# Pipeline to process PPMI MRI diffusion data

### Data
Data is downloaded in raw DICOM format from the PPMI website. The example selected here at random is a patient over 70 years old and getting a scan in year 4. 

### Processing
This pipeline is a record of the basic steps to preprocess the data before statistical analysis. The PPMI pre-processing pipeline is available at http://www.ppmi-info.org/wp-content/uploads/2011/12/DTI-processing-Pipeline3.pdf
1. Download the data into an input directory, setup workspace
1. Convert the DICOM format to NII format, compute bvals and bvecs
1. Correct each scan for signal distortions due to eddy currents and head motion
1. Rotate gradient directions in response to eddy current corrections. 
1. Remove non-brain tissue
1. Save the data into an output directory


This pipeline mainly follows this tutorial http://www.mccauslandcenter.sc.edu/crnl/sw/tutorial/html/dti.html

### Setup
1. Download the data, create directories

In [14]:
%%bash
ls -al  ../data/input/S264294 | head -6

total 145080
drwxrwx--- 1 root vboxsf    2144 Apr 17 08:27 .
drwxrwx--- 1 root vboxsf     128 Apr 17 09:49 ..
-rwxrwx--- 1 root vboxsf 2283182 Apr 12 20:06 PPMI_3107_MR_DTI_gated__br_raw_20150629091644684_2_S264294_I498877.dcm
-rwxrwx--- 1 root vboxsf 2283200 Apr 12 20:06 PPMI_3107_MR_DTI_gated__br_raw_20150629091645962_33_S264294_I498877.dcm
-rwxrwx--- 1 root vboxsf 2283196 Apr 12 20:05 PPMI_3107_MR_DTI_gated__br_raw_20150629091647410_20_S264294_I498877.dcm


In [15]:
%%bash
du -s -h ../data/input/S264294

142M	../data/input/S264294


In [17]:
%%bash
mkdir ../data/working/S264294
mkdir ../data/output/S264294

### Convert DICOM data to NII

In [19]:
%%bash
dcm2niix -o ../data/working/S264294 -z y  ../data/input/S264294

Chris Rorden's dcm2niiX version v1.0.20180328 GCC5.4.0 (64-bit Linux)
Found 65 DICOM image(s)
slices stacked despite varying acquisition numbers (if this is not desired please recompile)
Convert 65 DICOM as ../data/working/S264294/S264294_DTI_gated_20150508093306_12 (116x116x72x65)
compress: "/usr/bin/pigz" -n -f -6 "../data/working/S264294/S264294_DTI_gated_20150508093306_12.nii"
Conversion required 19.059444 seconds (1.493815 for core code).


In [20]:
%%bash
ls -al ../data/working/S264294

total 60192
drwxrwx--- 1 root vboxsf      192 Apr 17 10:19 .
drwxrwx--- 1 root vboxsf      128 Apr 17 10:08 ..
-rwxrwx--- 1 root vboxsf      322 Apr 17 10:19 S264294_DTI_gated_20150508093306_12.bval
-rwxrwx--- 1 root vboxsf     1815 Apr 17 10:19 S264294_DTI_gated_20150508093306_12.bvec
-rwxrwx--- 1 root vboxsf     2310 Apr 17 10:19 S264294_DTI_gated_20150508093306_12.json
-rwxrwx--- 1 root vboxsf 61001413 Apr 17 10:19 S264294_DTI_gated_20150508093306_12.nii.gz


In [25]:
%matplotlib inline
import nibabel as nib

img = nib.load("../data/working/S264294/S264294_DTI_gated_20150508093306_12.nii.gz")

In [27]:
print(img.header)

<class 'nibabel.nifti1.Nifti1Header'> object, endian='<'
sizeof_hdr      : 348
data_type       : b''
db_name         : b''
extents         : 0
session_error   : 0
regular         : b'r'
dim_info        : 57
dim             : [  4 116 116  72  65   0   0   0]
intent_p1       : 0.0
intent_p2       : 0.0
intent_p3       : 0.0
intent_code     : none
datatype        : int16
bitpix          : 16
slice_start     : 0
pixdim          : [-1.         1.9827586  1.9827586  2.         0.9        0.
  0.         0.       ]
vox_offset      : 0.0
scl_slope       : nan
scl_inter       : nan
slice_end       : 0
slice_code      : sequential increasing
xyzt_units      : 10
cal_max         : 0.0
cal_min         : 0.0
slice_duration  : 0.0
toffset         : 0.0
glmax           : 0
glmin           : 0
descrip         : b'TE=88;Time=101826.620;phase=1'
aux_file        : b''
qform_code      : scanner
sform_code      : scanner
quatern_b       : 0.0
quatern_c       : 1.0
quatern_d       : 0.0
qoffset_x       : 1

In [28]:
img.affine

array([[ -1.98275864,   0.        ,  -0.        , 114.99999237],
       [ -0.        ,   1.98275864,  -0.        , -93.45097351],
       [  0.        ,   0.        ,   2.        , -66.27710724],
       [  0.        ,   0.        ,   0.        ,   1.        ]])

### Eddy Correction

This takes a long time (~ 1 hour)

In [30]:
%%bash
fsl5.0-eddy_correct ../data/working/S264294/S264294_DTI_gated_20150508093306_12.nii.gz \
  ../data/working/S264294/S264294_DTI_gated_20150508093306_12_eddy 0

processing ../data/working/S264294/S264294_DTI_gated_20150508093306_12_eddy_tmp0000
processing ../data/working/S264294/S264294_DTI_gated_20150508093306_12_eddy_tmp0001
processing ../data/working/S264294/S264294_DTI_gated_20150508093306_12_eddy_tmp0002
processing ../data/working/S264294/S264294_DTI_gated_20150508093306_12_eddy_tmp0003
processing ../data/working/S264294/S264294_DTI_gated_20150508093306_12_eddy_tmp0004
processing ../data/working/S264294/S264294_DTI_gated_20150508093306_12_eddy_tmp0005
processing ../data/working/S264294/S264294_DTI_gated_20150508093306_12_eddy_tmp0006
processing ../data/working/S264294/S264294_DTI_gated_20150508093306_12_eddy_tmp0007
processing ../data/working/S264294/S264294_DTI_gated_20150508093306_12_eddy_tmp0008
processing ../data/working/S264294/S264294_DTI_gated_20150508093306_12_eddy_tmp0009
processing ../data/working/S264294/S264294_DTI_gated_20150508093306_12_eddy_tmp0010
processing ../data/working/S264294/S264294_DTI_gated_20150508093306_12_eddy_

### Brain Extration

In [31]:
%%bash
fsl5.0-bet ../data/working/S264294/S264294_DTI_gated_20150508093306_12_eddy.nii.gz \
    ../data/working/S264294/S264294_DTI_gated_20150508093306_12_eddy_brain -m

In [33]:
%%bash
cp ../data/working/S264294/S264294_DTI_gated_20150508093306_12_eddy_brain_mask.nii.gz \
    ../data/working/S264294/S264294_DTI_gated_20150508093306_12_eddy_brain.nii.gz \
    ../data/working/S264294/S264294_DTI_gated_20150508093306_12.bval \
    ../data/working/S264294/S264294_DTI_gated_20150508093306_12.bvec \
    ../data/working/S264294/S264294_DTI_gated_20150508093306_12.json \
    ../data/output/S264294

### Now we are ready to fit the stastical models to calculate diffusion tensors